In [5]:
import pandas as pd
import json
import openai
import inspect
from dotenv import load_dotenv
import os
load_dotenv("..\credentials.env",override=True)

True

In [24]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

In [10]:
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
subscription_key = os.environ["AZURE_OPENAI_API_KEY"]
deployment = "gpt-4o"

# Azure AI Search setup
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT"); # Add your Azure AI Search endpoint here
search_key = os.getenv("AZURE_SEARCH_KEY"); # Add your Azure AI Search admin key here
search_index_name = "northwind-vectorstore"

In [25]:
client = openai.AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",  
)

search_client = SearchClient(
     endpoint=search_endpoint,
     index_name=search_index_name,    
     credential=AzureKeyCredential(search_key)
 )

In [71]:
# Provide instructions to the model
GROUNDED_PROMPT="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
Use bullets if the answer has multiple points.
If the answer is longer than 3 sentences, provide a summary.
Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

#

In [72]:

def naive_rag(query):    
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="text_vector")
    search_results = search_client.search(  
        search_text=query,  
        vector_queries= [vector_query],
        select=["title", "chunk", "chunk_id"],
        top=1
    )

    search_results.get_count()
    
    #for result in search_results:  
        #print(result)
    #     print(f"Score: {result['@search.score']}")
    #     print(f"Chunk: {result['chunk']}")
    sources_formatted = "=================\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}' for document in search_results])
    sources_formatted
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
            }
        ],
        model=deployment
    )

    print(response.choices[0].message.content)

In [73]:
naive_rag("Does Northwind health plan covers eye exams?")

I don't know.


In [75]:
naive_rag("I am unable to listen to the sound, does northwind cover the test?")

Based on the provided sources, it is unclear whether the Northwind Standard plan specifically covers tests related to an inability to listen to sound. However, the plan includes exceptions such as not covering services that are not medically necessary or provided by health care providers not contracted with Northwind Health (Source: "Northwind_Standard_Benefits_Details.pdf"). Therefore, you should ensure that:

- The service is deemed medically necessary.
- The health care provider is contracted with Northwind Health.

If these conditions are met, you should send the relevant documentation, including test results and physician's notes, to Northwind Health and follow up to ensure your claim is being processed. 

(Source: "Northwind_Standard_Benefits_Details.pdf")


In [60]:
naive_rag("what should I do if I feel feverish?")

I don't know.


In [61]:
naive_rag("how can I improve my health?")

Based on the provided document, here are some tips to improve your health:

- Track your weight loss progress to stay motivated and on track.
- Seek support from friends, family, or a healthcare professional to remain accountable and motivated.

(Source: Northwind_Standard_Benefits_Details.pdf)


In [ ]:
naive_rag("Does Northwind plans cover testing cholestrol levels?")
